# 순서도

1. 이미지 경로 설정

2. 오디오 길이 측정

3. vfx 설정(각 함수에 맞게 오디오 설정)

4. 전환 효과 설정



In [2]:
!pip install moviepy

     ---------------------------------------- 0.0/388.3 kB ? eta -:--:--
     --------- ----------------------------- 92.2/388.3 kB 2.6 MB/s eta 0:00:01
     -------------------------------------  378.9/388.3 kB 4.7 MB/s eta 0:00:01
     -------------------------------------- 388.3/388.3 kB 4.8 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/313.4 kB ? eta -:--:--
   ---------------------------------------- 313.4/313.4 kB 9.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/22.6 MB ? eta -:--:--
   - -------------------------------------- 0.7/22.6 MB 14.8 MB/s eta 0:00:02
   -- ------------------------------------- 1.4/22.6 MB 15


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: C:\Users\fhsdn_6m\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [1]:
import glob, cv2, random, tqdm, src, moviepy, os
import numpy as np
from moviepy.editor import AudioFileClip, concatenate_videoclips, ImageSequenceClip, concatenate_audioclips, CompositeAudioClip

In [2]:
root_path = 'D:/SKT'

In [3]:
subtitle = [['죽음의 순간, 하나의 삶이 끝나고', '또 다른 삶이 시작된다.'],
 ['억울한 죽음 뒤, 재벌가의 막내아들로', '다시 태어난 윤현우.'],
 ['가장 깊은 절망에서 빛나는 희망의 빛,', '진도준으로의 거듭남.'],
 ['한 아이의 대담한 목표,', '할아버지의 전폭적인 지지 아래 시작된다.'],
 ['분당 땅, 한번의 선택이 역사를 잇는', '부의 시작이 된다.'],
 ['목장에 심겨진 소년 진도준의 꿈꾸는 미래,', '경제적 기적의 서막.'],
 ['미래를 아는 소년,', '그의 선택이 역사를 바꾼다.'],
 ['구글과의 만남,', '진도준의 꿈을 현실로 만드는 결정적인 순간.'],
 ['조의 권력을 한 손에,', "'미라클 인베스트먼트'의 창립."],
 ['순양그룹을 향한 그의 움직임,', '한 발 한 발 앞으로.'],
 ['숙적들을 무너뜨리기 위한 진도준의 냉철한', '계획.'],
 ['과거를 바로잡고 미래를 차지하려는', '끝없는 갈망.'],
 ['더 자세한 내용은',"'재벌집 막내아들' 에서!"]]

effects = ['페이드 인',
 '눈 깜빡임 효과',
 '줌 아웃',
 '줌 인',
 '페이드 아웃',
 '눈 깜빡임 효과',
 '페이드 인',
 '줌 인',
 '배경 없이 나레이션만',
 '줌 아웃',
 '페이드 아웃',
 '줌 인']

In [4]:
selected_image = [1,0,1,0,1,0,1,0,0,1,0,1]
img_path = []
for i,j in enumerate(selected_image):
    filename= f"{root_path}/image/{i:02d}-{j:02d}.png"
    assert os.path.exists(filename)
    img_path.append(filename)
img_path,len(img_path)

(['D:/SKT/image/00-01.png',
  'D:/SKT/image/01-00.png',
  'D:/SKT/image/02-01.png',
  'D:/SKT/image/03-00.png',
  'D:/SKT/image/04-01.png',
  'D:/SKT/image/05-00.png',
  'D:/SKT/image/06-01.png',
  'D:/SKT/image/07-00.png',
  'D:/SKT/image/08-00.png',
  'D:/SKT/image/09-01.png',
  'D:/SKT/image/10-00.png',
  'D:/SKT/image/11-01.png'],
 12)

In [5]:
audio_path = sorted(glob.glob(f"{root_path}/audio/*"))
len(audio_path)

26

In [6]:
bgm = AudioFileClip(f"{root_path}/BGM.wav")

In [7]:


def floor(lst:list):
    for i in range(len(lst)):
        for j in range(len(lst[i])):
            lst[i][j] = int(lst[i][j])
    return lst


fps = 60 # 영상 프레임

imgs = [] # 이미지
audios = [] # 오디오

audio_frames = [] # 오디오별 길이

for p in img_path:
    imgs.append(cv2.cvtColor(cv2.imread(p),cv2.COLOR_BGR2RGB))
imgs.append(np.zeros_like(imgs[-1],dtype=np.uint8))

idx = 0 
for i in range(len(subtitle)):
    audios.append([])
    audio_frames.append([])
    for j in range(len(subtitle[i])):
        audios[i].append(AudioFileClip(audio_path[idx]))
        audio_frames[i].append(audios[i][j].duration*fps)
        idx = idx + 1

audio_frames = floor(audio_frames)

print("이미지 수 :",len(imgs),", 나레이션 수 :",len(audios))
transition_frames = [30,30]
transition_lst = [src.transition.Slide,src.transition.Blending]
transition = []


frames = [sum(audio_frames[0])] # 
for i in range(1,len(imgs)):
    idx = random.randint(0,len(transition_lst)-1)
    transition.append(transition_lst[idx](transition_frames[idx]))
    frames.append(transition_frames[idx])
    frames.append(sum(audio_frames[i]))
print(frames)

# 마지막 나레이션 장면 전환 효과 (더 자세한 내용은 '~~' 에서!)
transition[-1] = transition_lst[1]

이미지 수 : 13 , 나레이션 수 : 13
[226, 30, 229, 30, 241, 30, 263, 30, 217, 30, 259, 30, 175, 30, 232, 30, 208, 30, 189, 30, 201, 30, 204, 30, 162]


In [8]:
vfx_lst = [
    "없음",
    "페이드 인",
    "페이드 아웃",
    "줌 인",
    "줌 아웃",
    "눈 깜빡임 효과",
    "배경 없이 나레이션만"
]


def identity(video:src.Video):
    return video

vfx = [
    identity,
    src.effect.Fade_in(0,60),
    src.effect.Fade_out(-60),
    src.effect.Zoom_in(0),
    src.effect.Zoom_out(0),
    src.effect.Blink(0,60),
    identity
]
vfx_idx = []
for j,e in enumerate(effects):
    for i in range(len(vfx_lst)):
        if e.find(vfx_lst[i]) != -1:
            vfx_idx.append(i)
            break

# 마지막 나레이션 영상 효과 (더 자세한 내용은 '~~' 에서!)
vfx_idx.append(6)
vfx_idx,len(vfx_idx)

([1, 5, 4, 3, 2, 5, 1, 3, 6, 4, 2, 3, 6], 13)

In [9]:
def apply_subtitle(video:src.Video,idx:int):
    start_frame = 0
    ret = video
    if vfx_idx[idx] == 6:
        for i in range(len(subtitle[idx])):
            ret = src.effect.Narration(subtitle[idx][i],start_frame=start_frame,end_frame=start_frame+audio_frames[idx][i])(ret)
            start_frame = start_frame + audio_frames[idx][i]
    else:
        for i in range(len(subtitle[idx])):
            ret = src.effect.Subtitle(subtitle[idx][i],start_frame=start_frame,end_frame=start_frame+audio_frames[idx][i],color=(255,255,255),bg_color=(0,0,0))(ret)
            start_frame = start_frame + audio_frames[idx][i]
    return ret

videos = [apply_subtitle(vfx[vfx_idx[0]](src.im2vid(imgs[0],sum(audio_frames[0]))),0)]
for i in tqdm.tqdm(range(1,len(imgs))):
    x1 = videos[-1]
    x2 = apply_subtitle(vfx[vfx_idx[i]](src.im2vid(imgs[i],sum(audio_frames[i]))),i)
    videos.append(transition[i-1](x1,x2))
    videos.append(x2)

100%|██████████| 12/12 [02:40<00:00, 13.38s/it]


In [10]:
videos

[Frame shape(h,w):(1792, 1024), Num_frames:226, FPS:60, Duration:3.7666666666666666 Sec,
 Frame shape(h,w):(1792, 1024), Num_frames:30, FPS:60, Duration:0.5 Sec,
 Frame shape(h,w):(1792, 1024), Num_frames:229, FPS:60, Duration:3.816666666666667 Sec,
 Frame shape(h,w):(1792, 1024), Num_frames:30, FPS:60, Duration:0.5 Sec,
 Frame shape(h,w):(1792, 1024), Num_frames:241, FPS:60, Duration:4.016666666666667 Sec,
 Frame shape(h,w):(1792, 1024), Num_frames:30, FPS:60, Duration:0.5 Sec,
 Frame shape(h,w):(1792, 1024), Num_frames:263, FPS:60, Duration:4.383333333333334 Sec,
 Frame shape(h,w):(1792, 1024), Num_frames:30, FPS:60, Duration:0.5 Sec,
 Frame shape(h,w):(1792, 1024), Num_frames:217, FPS:60, Duration:3.6166666666666667 Sec,
 Frame shape(h,w):(1792, 1024), Num_frames:30, FPS:60, Duration:0.5 Sec,
 Frame shape(h,w):(1792, 1024), Num_frames:259, FPS:60, Duration:4.316666666666666 Sec,
 Frame shape(h,w):(1792, 1024), Num_frames:30, FPS:60, Duration:0.5 Sec,
 Frame shape(h,w):(1792, 1024), 

In [11]:
video_lst = []


for i in tqdm.tqdm(range(len(videos))):
    vid = ImageSequenceClip(videos[i].video, fps=fps)
    if i != len(videos):
        if i%2 == 0:
            video_lst.append(vid.set_audio(concatenate_audioclips(audios[i//2])))
        else :
            video_lst.append(vid)
    
video_clip = concatenate_videoclips(video_lst)
new_audio = CompositeAudioClip([video_clip.audio,bgm.audio_loop(duration=video_clip.duration).volumex(1.5)])
video_clip.audio = new_audio


# 비디오를 저장합니다.
video_clip.write_videofile("test.mp4")

100%|██████████| 25/25 [00:00<00:00, 448.70it/s]

Moviepy - Building video test.mp4.
MoviePy - Writing audio in testTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video test.mp4



Moviepy - Done !
Moviepy - video ready test.mp4
